### Alice möchte Bob 10000 satoshis senden welche Bob anschließend wieder ausgibt. Bob gibt Alice dabei verschiedene Möglichkeiten wie die Satoshis für ihn gelocked werden.

Die ersten Möglichkeit ist das Versenden der Satoshis an einen öffentlichen Schlüssel welcher durch einen von Bob kontrollierten privaten Schlüssel erzeugt wurde. Dieser Fall wird **P2PK-Transaktion** genannt.

In [1]:
#bitcoind -regtest -daemon -fallbackfee=0.0000025 -> startet bitcoin in regtest mode
#bcr loadwallet alice (oder bcr createwallet alice)
#bcr loadwallet bob (oder bcr createwallet bob)
#bcr -rpcwallet=alice getnweaddress "" legacy -> gibt die addresse zur Vorbereitung von welcher dann die P2PK-Locking Transaktion ausgeht
#bcr -rpcwallet=alice sendtoaddress <address> <amount in btc>
#bcr -rpcwallet=alice generatetoaddress <amount of blocks> <address>
from io import BytesIO

from src.transaction import CTx, CTxIn, CTxOut
from src.script import Script, p2pk_script
from src.ecdsa import PrivateKey

**1.** In der ersten Transaktion werden von Alice die 10000 Satoshis an den von Bob kontrollierten öffentlichen Schlüssel versendet. Um das notwendige Transaktionsformular ausfüllen zu können, benötigt Alice folgende Informationen. 

In [3]:
#Transaktionsinputdaten
#bcr -rpcwallet=alice dumpprivkey <alice address> -> gibt den zu address gehörigen privaten Schlüssel in WIF
alice_private_key_wif_format = "cRzut5ygumUoZfztPwdjUq9n5ycqCkbShm8b7eSrWkudDWXesvQ2"
alice_private_key = PrivateKey.convert_wif_format(alice_private_key_wif_format)
alice_private_key_int_format = alice_private_key.get_private_key_int()
alice_public_key = alice_private_key.get_public_key()

#bcr -rpcwallet=alice listunspent -> listet alle utxo's von alice auf
#bcr -rpcwallet=alice gettransaction <tx_id> -> gibt das "Überweisungsformular" der Transaktion mit der angegebenen ID zurück.
alice_previous_tx_id_to_spent = "338ec01d36ddeebc9dc333c843afe9c954d758e3bc8c102bf7e0ba3cb70fbc2f"
alice_previous_tx_index_to_spent = 1
alice_previous_script_pub_key_to_spent = "76a91498e38c6956e6c320d2eee5a8d73502bd2abf134588ac"

#Transaktionsoutputdaten
alice_amount_to_spent = 10000
bob_public_key = PrivateKey(70391367530871559720919750735257129489246656383889830217409950893950252925507).get_public_key()

**1.1** Nun kann Alice die Lockingtransaktion erzeugen. Um die Satoshis an Bob's öffentlichen Schlüssel zu senden, muss Alice im Output der Transaktion ein scriptPubKey der Form: <Bob's public Key> + CHECKSIG generieren.

In [4]:
transaction_input = CTxIn(bytes.fromhex(alice_previous_tx_id_to_spent), alice_previous_tx_index_to_spent, script_sig=Script())

alice_script_pubkey = p2pk_script(bob_public_key.sec_format())
transaction_output = CTxOut(alice_amount_to_spent, alice_script_pubkey)

alice_locking_transaction = CTx(1, [transaction_input], [transaction_output], 0xffffffff, is_testnet=True, is_segwit=False)

print(f"Alice unsignierte P2PK Lockingtransaktion: {alice_locking_transaction.serialize_transaction().hex()}")

Alice unsignierte P2PK Lockingtransaktion: 01000000012fbc0fb73cbae0f72b108cbce358d754c9e9af43c833c39dbceedd361dc08e330100000000ffffffff011027000000000000232103982f610d344db57d441e594d4d4f5001fe1369081a42f8350b49be562e94dfb4acffffffff


**1.1.1** Nun muss Alice die Lockingtransaktion noch signieren. D.h. sie muss nachweisen, dass sie aus der vorhergehenden Transaktion berechtigt ist die 10000 Satoshis auszugeben.

In [6]:
alice_script_sig = Script().parse_script(BytesIO(bytes.fromhex(f"{hex(len(alice_previous_script_pub_key_to_spent)//2)[2:]}{alice_previous_script_pub_key_to_spent}")))
alice_locking_transaction.sign_transaction(0, alice_private_key_int_format, alice_script_sig)

#bcr -rpcwallet=alice testmempoolaccept '["tx"]' -> prüft ob die Transaktion alle Checks für die Aufnhame in den Mempool besteht
#bcr -rpcwallet=alice decoderawtransaction <tx> -> decodiert die serilaisierte Transaktion ins "Überweisungsformular"
#bcr -rpcwallet=alice sendrawtransaction <tx> -> übermittelt schließlich die Transaktion im Netzwerk so dass sie von minern für einen block aus dem mempool aufgenommen werden kann

print(f"Alice signierte P2PK Lockingtransaktion: {alice_locking_transaction.serialize_transaction().hex()}")

Alice signierte P2PK Lockingtransaktion: 01000000012fbc0fb73cbae0f72b108cbce358d754c9e9af43c833c39dbceedd361dc08e33010000006b483045022100a13b88c11ce1a923e4fa92c92d7ea7d1ae80ceab5c5e256896f5161dbcdc6a2b022030bb70a3b551db24cded3be517cce086f23ebe93d5b6e25d90d614c28350542401210333fe3326673ccd0eb62eb24b18115b575474dcd58cf7c9804b601f1afe36ecf8ffffffff011027000000000000232103982f610d344db57d441e594d4d4f5001fe1369081a42f8350b49be562e94dfb4acffffffff


**2.** Nun möchte Bob die an seinen öffentlichen Schlüssel gesendeten Satoshis weiter versenden. Dazu muss Bob im Inputfeld (in dem auf den eben von Alice erzeugten Output referenziert wird), zum Beweis das er  berechtigt ist aen Output auszugeben lediglich eine Signatur mit Bob's privatem Schlüssel stehen. Damit Bob das notwendige Transaktionsformular ausfüllen kann benötigt er folgende Informationen.

In [7]:
#Transaktionsinputdaten
#bcr -rpcwallet=bob dumpprivkey <bob address>
bob_private_key_wif_format = "cSoDYWXxTwTbnWBpTsgWiaJbD4ZTLpJ51nHppZRFKtCK418ERJEo"
bob_private_key = PrivateKey.convert_wif_format(bob_private_key_wif_format)
bob_private_key_int_format = bob_private_key.get_private_key_int()
bob_public_key = bob_private_key.get_public_key()

#bcr -rpcwallet=bob listunspent
bob_previous_tx_id_to_spent = 'd0cd7941815b54a4196df5a44e4428a520bf672522f7a1bbd61b9ba22e66a8cc'
bob_previous_tx_index_to_spent = 0
bob_previous_script_pub_key_to_spent = "2103982f610d344db57d441e594d4d4f5001fe1369081a42f8350b49be562e94dfb4ac"

#Transaktionsoutputdaten
bob_amount_to_spent = 9000
#bob's reciever address is alice public key

**2.1** Nun kann Bob die Transaktion erstellen um die eben von Alice erhaltenen Satoshis weiter zu versenden.

In [8]:
transaction_input = CTxIn(bytes.fromhex(bob_previous_tx_id_to_spent), bob_previous_tx_index_to_spent, script_sig=Script())

script_pubkey = p2pk_script(alice_public_key.sec_format())
transaction_output = CTxOut(bob_amount_to_spent, script_pubkey)

bob_spending_transaction = CTx(1, [transaction_input], [transaction_output], 0xffffffff, is_testnet=True, is_segwit=False)

print(f"Bob unsignierte P2PK Spendingtransaktion: {bob_spending_transaction.serialize_transaction().hex()}")


Bob unsignierte P2PK Spendingtransaktion: 0100000001cca8662ea29b1bd6bba1f7222567bf20a528444ea4f56d19a4545b814179cdd00000000000ffffffff01282300000000000023210333fe3326673ccd0eb62eb24b18115b575474dcd58cf7c9804b601f1afe36ecf8acffffffff


**2.2** Nun kann Bob diese Transaktion mit seinem privaten Schlüssle signieren und versenden.

In [9]:
bob_script_sig = Script().parse_script(BytesIO(bytes.fromhex(f"{hex(len(bob_previous_script_pub_key_to_spent)//2)[2:]}{bob_previous_script_pub_key_to_spent}")))
bob_spending_transaction.sign_transaction(0, bob_private_key_int_format, bob_script_sig)

#bcr -rpcwallet=bob testmempoolaccept '["tx"]'
#bcr -rpcwallet=bob decodetransaction <tx>
#bcr -rpcwallet=bob sendrawtransaction <tx>

print(f"Bob signierte P2PK Spendingtransaktion: {bob_spending_transaction.serialize_transaction().hex()}")

Bob signierte P2PK Spendingtransaktion: 0100000001cca8662ea29b1bd6bba1f7222567bf20a528444ea4f56d19a4545b814179cdd000000000494830450221009429f5119f1fce8a0b728eeddca4949ddeeed9ac2f12ff643de7b44b9b3bd1e602206f660ee4e655e51a017e38a9c53bb42f1a4d1df0d7e5314e5c0b529d6d5e6abb01ffffffff01282300000000000023210333fe3326673ccd0eb62eb24b18115b575474dcd58cf7c9804b601f1afe36ecf8acffffffff
